In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import layers
import tensorflow as tf
IMAGE_ORDERING = 'channels_last'
tf.executing_eagerly()
BATCH_SIZE=128

In [ ]:
#load data
import numpy as np
Xdata=np.load("Trainingdata.npy")
# mindata=Xdata.min(axis=(0,1,2))
# maxdata=Xdata.max(axis=(0,1,2))
# Xdata=(Xdata-mindata)/(maxdata-mindata)
Ydata=np.load("traininglabel.npy")[:,:,:,2:]
# Y_area=np.load("labels_area_density.npy")
#normalizer.adapt(Xdata)
#Ydata=np.expand_dims(np.argmax(Y_sus,axis=-1),axis=-1)
mindata=Xdata.min(axis=(0,1,2))
maxdata=Xdata.max(axis=(0,1,2))
Xdata=(Xdata-mindata)/(maxdata-mindata)

In [ ]:
Ydata.shape
Xdata=Xdata.reshape((170880,32,32,13))
Ydata=Ydata.reshape((170880,1))

#load data
import numpy as np
Xdata=np.load("Covariates_full.npy")
Y_sus=np.load("labels_susceptibility.npy")
Y_area=np.load("labels_area_density.npy")
#normalizer.adapt(Xdata)
Ydata=Y_area#np.expand_dims(np.argmax(Y_sus,axis=-1),axis=-1)

In [ ]:
idx=np.where(Ydata>0)[0]
Ydata=Ydata[idx]
Xdata=Xdata[idx]
Ydata.shape,Xdata.shape

In [ ]:
def get_regression(n_classes=1,input_height=32,  input_width=32, channels=13):

    assert input_height % 32 == 0
    assert input_width % 32 == 0
    
    if IMAGE_ORDERING == 'channels_first':
        img_input = Input(shape=(channels, input_height, input_width))
    elif IMAGE_ORDERING == 'channels_last':
        img_input = Input(shape=(input_height, input_width, channels))

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    if IMAGE_ORDERING == 'channels_first':
        MERGE_AXIS = 1
    elif IMAGE_ORDERING == 'channels_last':
        MERGE_AXIS = -1
        
    x = ZeroPadding2D((3, 3), data_format=IMAGE_ORDERING)(img_input)
    x = Conv2D(64, (4, 4), data_format=IMAGE_ORDERING,
               strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), data_format=IMAGE_ORDERING, strides=(2, 2))(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_dense1')(x)
    x = MaxPooling2D((3, 3), data_format=IMAGE_ORDERING, strides=(2, 2))(x)
    x=  Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    o1 = Dense(1, activation='relu')(x)
    model = Model(inputs=[img_input], outputs=[o1])
    return model

In [ ]:
model=get_regression()

In [ ]:
model.load_weights("models_checkpoint_regression_V2.5")
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3,decay_steps=20000,decay_rate=0.9)
import keras.backend as K
opt = Adam(learning_rate=1e-4)
def regressionloss(y_true, y_pred):
    y_true_flat=K.flatten(y_true)
    y_pred_flat=K.flatten(y_pred)
    mask=tf.where(y_pred_flat==0.,False,True)
    count_nonzero=tf.reduce_sum(tf.cast(mask, tf.float32))
    return tf.math.divide_no_nan(tf.reduce_sum(tf.math.square(tf.math.subtract(y_true_flat,y_pred_flat))),count_nonzero)

model.compile(optimizer=opt, 
              loss=tf.keras.losses.MeanAbsoluteError(), 
              metrics=tf.keras.metrics.RootMeanSquaredError())

In [ ]:
NUMBER_EPOCHS = 500
filepath="models_checkpoint_regression_V2.3"
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch",
    options=None
)
import os,datetime
logdir = os.path.join("logs_regression","Type_2_v3"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


def train(model, xdata,ydata):
    hist = model.fit(x=xdata,
                     y=ydata,
                     epochs=NUMBER_EPOCHS,
                     validation_split=0.3, 
                     batch_size=BATCH_SIZE,#auto validate using 30% of random samples at each epoch
                     verbose=1, callbacks=[model_checkpoint_callback,tensorboard_callback]
                    )
    return hist

In [ ]:
history=train(model,Xdata,Ydata)

In [ ]:
Ydata_predicted=model.predict(Xdata)

In [ ]:
import seaborn as sns; sns.set_theme(color_codes=True)
import scipy as sp
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(5,5)})
ax = sns.regplot(x=Ydata, y=Ydata_predicted,fit_reg=False)

r, p = sp.stats.pearsonr(Ydata.flatten(), Ydata_predicted.flatten())
ax.text(.05, .8, 'r={:.2f}'.format(r),
        transform=ax.transAxes)
xx=np.arange(0,60)
yy=np.arange(0,60)
sns.lineplot( x=xx, y=yy, color='r', ax=ax)
ax.set(xlabel='Observed',ylabel='Predicted',title='Obs vs Pred Area Density' )
ax.set_xlim((0,60))
ax.set_ylim((0,60))

In [ ]:
ax.get_figure().savefig("regression_dat_v3.png",dpi=500) 

In [ ]:
import seaborn as sns
def plot_hist(data):
    sns.kdeplot(
   data=data, 
   fill=True, common_norm=False, palette="crest",
   alpha=.5, linewidth=0,
)